# I. Import library and load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv("processed data.csv")
data = data.drop(columns = ['Unnamed: 0'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Vision,Volleys,age,name,overall,potential,preferred_foot,position common,market value,wage value
0,34,2983,0.06,0.00,0.15,0.03,Ligue 1,0.54,37,81,...,45,24,33,Yunis Abdelhamid,76,76,Left,defender,3600000.0,23000.0
1,31,2462,0.04,0.00,0.44,0.11,Ligue 1,0.66,50,79,...,65,38,21,Salis Abdul Samed,70,75,Right,midfielder,2200000.0,7000.0
2,34,2956,0.00,0.06,0.27,0.00,Ligue 1,0.91,77,76,...,66,37,28,Laurent Abergel,75,75,Right,midfielder,4900000.0,18000.0
3,24,726,0.00,0.12,0.37,0.00,Bundesliga,2.22,81,39,...,44,53,22,Dickson Abiama,68,76,Right,striker,2700000.0,9000.0
4,30,2536,0.14,0.00,0.07,0.04,Serie A,0.57,64,77,...,65,39,33,Francesco Acerbi,83,83,Left,defender,17500000.0,75000.0


# II. Convert category data into numeric data and remove columns that don't related in build model

In [3]:
# Comp = name of league
data['Comp'].replace(data['Comp'].unique(),[1, 3, 2, 4, 5], inplace=True)
# drop name column and preferred_foot
data = data.drop(columns = ['name','preferred_foot'])

In [4]:
#convert the categorical variables to dummies
data = pd.get_dummies(data) 
# log tranformation for market vakue
data['market value'] = np.log(data['market value'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Volleys,age,overall,potential,market value,wage value,position common_defender,position common_goalkeeper,position common_midfielder,position common_striker
0,34,2983,0.06,0.00,0.15,0.03,1,0.54,37,81,...,24,33,76,76,15.096444,23000.0,1,0,0,0
1,31,2462,0.04,0.00,0.44,0.11,1,0.66,50,79,...,38,21,70,75,14.603968,7000.0,0,0,1,0
2,34,2956,0.00,0.06,0.27,0.00,1,0.91,77,76,...,37,28,75,75,15.404746,18000.0,0,0,1,0
3,24,726,0.00,0.12,0.37,0.00,3,2.22,81,39,...,53,22,68,76,14.808762,9000.0,0,0,0,1
4,30,2536,0.14,0.00,0.07,0.04,2,0.57,64,77,...,39,33,83,83,16.677711,75000.0,1,0,0,0


# III. Split training and test

In [5]:
data_build = data.copy()
y = data_build['market value']
x = data_build.drop(columns =['market value'])

# Standardization
def standard_scale(x):
    convert = StandardScaler().fit(x)
    return convert.transform(x)
#x = standard_scale(x)
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.33, random_state = 3 )

# IV.Emsemble models : LightGBM

In [6]:
import lightgbm as lgb

In [10]:
regressor = lgb.LGBMRegressor()
params = {
    'boosting_type':['gbdt', 'goss','dart' ],
    'n_estimators': range(100, 220, 50),
    'learning_rate': [x/100 for x in range(1, 21, 2)],
    'reg_lambda' :[0, 0.2, 0.4, 0.6, 0.8, 1],
    'max_depth':[int(x) for x in range(3, 17, 2)]
}
scoring_fnc = 'neg_mean_squared_error'
grid = GridSearchCV(estimator=regressor, param_grid=params, scoring= scoring_fnc, cv= 5 , verbose= 5)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 3780 candidates, totalling 18900 fits
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0;, score=-0.279 total time=   0.0s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0;, score=-0.300 total time=   0.0s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0;, score=-0.345 total time=   0.0s
[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0;, score=-0.303 total time=   0.0s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0;, score=-0.312 total time=   0.0s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0.2;, score=-0.280 total time=   0.0s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=100, reg_lambda=0.2;, score=-0.301 total time=   0.0s
[CV 3/5] END 

[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0;, score=-0.093 total time=   0.0s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0;, score=-0.094 total time=   0.0s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0.2;, score=-0.081 total time=   0.0s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0.2;, score=-0.090 total time=   0.0s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0.2;, score=-0.119 total time=   0.0s
[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0.2;, score=-0.094 total time=   0.0s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estimators=200, reg_lambda=0.2;, score=-0.095 total time=   0.0s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=3, n_estim

[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.2;, score=-0.110 total time=   0.1s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.2;, score=-0.113 total time=   0.1s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.2;, score=-0.142 total time=   0.1s
[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.2;, score=-0.120 total time=   0.1s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.2;, score=-0.122 total time=   0.1s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.4;, score=-0.112 total time=   0.2s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_estimators=150, reg_lambda=0.4;, score=-0.114 total time=   0.1s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=5, n_e

[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.2;, score=-0.244 total time=   0.1s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.2;, score=-0.255 total time=   0.1s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.4;, score=-0.227 total time=   0.1s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.4;, score=-0.246 total time=   0.1s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.4;, score=-0.281 total time=   0.1s
[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.4;, score=-0.246 total time=   0.1s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=100, reg_lambda=0.4;, score=-0.258 total time=   0.1s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_e

[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.4;, score=-0.055 total time=   0.3s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.4;, score=-0.056 total time=   0.3s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.4;, score=-0.084 total time=   0.3s
[CV 4/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.4;, score=-0.065 total time=   0.4s
[CV 5/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.4;, score=-0.064 total time=   0.3s
[CV 1/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.6;, score=-0.056 total time=   0.3s
[CV 2/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_estimators=200, reg_lambda=0.6;, score=-0.056 total time=   0.4s
[CV 3/5] END boosting_type=gbdt, learning_rate=0.01, max_depth=7, n_e

...

[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.4;, score=-153.340 total time=   0.2s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.4;, score=-21.346 total time=   0.1s
[CV 5/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.4;, score=-153.819 total time=   0.1s
[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.6;, score=-626.826 total time=   0.2s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.6;, score=-372.966 total time=   0.1s
[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.6;, score=-15.012 total time=   0.1s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=11, n_estimators=150, reg_lambda=0.6;, score=-372.333 total time=   0.1s
[CV 5/5] END boosting_type=dart, learning_rate=0.1

[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.6;, score=-31.572 total time=   0.1s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.6;, score=-424.565 total time=   0.1s
[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.6;, score=-430.511 total time=   0.1s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.6;, score=-423.695 total time=   0.1s
[CV 5/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.6;, score=-431.958 total time=   0.1s
[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.8;, score=-9.810 total time=   0.1s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=100, reg_lambda=0.8;, score=-2.846 total time=   0.1s
[CV 3/5] END boosting_type=dart, learning_rate=0.19, 

[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.6;, score=-17.399 total time=   0.3s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.6;, score=-174.410 total time=   0.2s
[CV 5/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.6;, score=-19.359 total time=   0.2s
[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.8;, score=-299.477 total time=   0.2s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.8;, score=-34.565 total time=   0.1s
[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.8;, score=-4.962 total time=   0.2s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=13, n_estimators=200, reg_lambda=0.8;, score=-98.261 total time=   0.2s
[CV 5/5] END boosting_type=dart, learning_rate=0.19, m

[CV 5/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.6;, score=-320.208 total time=   0.1s
[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.8;, score=-43.112 total time=   0.2s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.8;, score=-369.914 total time=   0.1s
[CV 3/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.8;, score=-0.971 total time=   0.1s
[CV 4/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.8;, score=-31.092 total time=   0.1s
[CV 5/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=0.8;, score=-45.437 total time=   0.1s
[CV 1/5] END boosting_type=dart, learning_rate=0.19, max_depth=15, n_estimators=150, reg_lambda=1;, score=-702.473 total time=   0.1s
[CV 2/5] END boosting_type=dart, learning_rate=0.19, ma

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'boosting_type': ['gbdt', 'goss', 'dart'],
                         'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.11,
                                           0.13, 0.15, 0.17, 0.19],
                         'max_depth': [3, 5, 7, 9, 11, 13, 15],
                         'n_estimators': range(100, 220, 50),
                         'reg_lambda': [0, 0.2, 0.4, 0.6, 0.8, 1]},
             scoring='neg_mean_squared_error', verbose=5)

In [11]:
# get best parameter
grid.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.11,
 'max_depth': 3,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 200,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.2,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [29]:
# Training model with hyperparameter turning
model = lgb.LGBMRegressor( boosting_type = 'gbdt', n_estimators = 200, learning_rate = 0.11,
                          reg_lambda  = 0.2, max_depth = 3)
model.fit(x_train, y_train)
#predict
y_pre = model.predict(x_test) 

print('r2 score :', r2_score(y_pre, y_test))
print('Root mean squared error: ' ,(mean_squared_error(y_pre,y_test )**(1/2)))

r2 score : 0.9933080232668359
Root mean squared error:  0.10288277769483303
